### 0. Imports

### 1. Introduction to this notebook

# 2. Transformation

## 2.3 Activities

In [36]:
import pandas as pd
import numpy as np

In [121]:
activities_df = pd.read_parquet("../data/accommodations/90_day_availability.parquet")

In [ ]:
activities_df

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,None,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2024-11-02 14:13:58.778257,2024-12-03,2024-12-09,Tour de los misterios y leyendas de Barcelona,En esta ruta nocturna por Barcelona descubrire...,www.civitatis.com/es/barcelona/tour-misterios-...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/tour-mist...,None,None,1h 30m,41.387471,2.172001,4.20,EUR,Visitas guiadas y free tours,Español
260,2024-11-02 14:13:58.779258,2024-12-03,2024-12-09,Entrada al Spotify Camp Nou,Si queréis visitar el estadio del Fútbol Club ...,www.civitatis.com/es/barcelona/camp-nou-experi...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/camp-nou-...,None,None,1h 30m,41.380053,2.121457,2.91,EUR,Entradas,Español
261,2024-11-02 14:13:58.780262,2024-12-03,2024-12-09,Free tour de los misterios y leyendas del Barr...,Acompañadnos en este free tour por Barcelona e...,www.civitatis.com/es/barcelona/free-tour-miste...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/free-tour...,None,None,2 horas,41.3838057,2.1757634,2.00,EUR,Visitas guiadas y free tours,Español
262,2024-11-02 14:13:58.781265,2024-12-03,2024-12-09,"Excursión a Gerona, Figueras y Museo Dalí",En esta excursión a Gerona y Figueras conocere...,www.civitatis.com/es/barcelona/excursion-museo...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/excursion...,None,None,9h - 10h 45m,41.3940811,2.1826233,23.70,EUR,Excursiones de un día,Español


### 2.3.1 Cleaning neccessary

1. Remove duplicates
2. Keep only the good image: The second if it has 2, otherwise the first.
3. Take unique, non null and non zero latitudes and longitudes, to convert to address.
4. Format the available dates and times

Careful as some non-available times might mean they do not apply, i.e. tickets.

##### 2.3.1.1 Remove duplicates

By activity_name and activity_date_range_start

In [123]:
activities_df[activities_df.duplicated(subset=["activity_date_range_start","activity_name"],keep=False)].sort_values(by="activity_name").head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
226,2024-11-02 14:13:58.688766,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 09, 03, 05, 08, 07]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
235,2024-11-02 14:13:58.724155,2024-12-03,2024-12-09,Barcelona Card Express,Con la tarjeta Barcelona Card Express obtendrá...,www.civitatis.com/es/barcelona/barcelona-card-...,www.civitatis.com/f/espana/barcelona/barcelona...,None,"[04, 06, 07, 05, 03, 08, 09]","[[0:00], [0:00], [0:00], [0:00], [0:00], [0:00...",2 días,0,0,5.40,EUR,Entradas,None
234,2024-11-02 14:13:58.706054,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 08, 09, 06, 04, 05, 03]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
243,2024-11-02 14:13:58.738170,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao El Duende,Asistiendo a un espectáculo flamenco en el Tab...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[07, 04, 06, 09, 03, 05, 08]","[[19:00, 20:15], [19:00, 20:15], [19:00, 20:15...",50 minutos,41.3789078,2.1747321,6.75,EUR,Espectáculos,None
237,2024-11-02 14:13:58.730169,2024-12-03,2024-12-09,Espectáculo flamenco en el Tablao de Carmen,¿Queréis experimentar lo mejor de la cultura e...,www.civitatis.com/es/barcelona/espectaculo-fla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/espana/barcelona/espectacu...,"[05, 09, 04, 07, 03]","[[], [], [], [], []]",1 hora,41.3689218,2.1488681,17.00,EUR,Espectáculos,None


In [124]:
activities_df.drop_duplicates(subset=["activity_date_range_start","activity_name"], inplace=True)

#### 2.3.1.2 Keep only the good image: The second if it has 2, otherwise the first.

In [125]:
activities_df["image"] = np.where(~activities_df["image2"].isna(), activities_df["image2"], activities_df["image"])
activities_df.drop(columns="image2",inplace=True)

In [ ]:
activities_activities_activities_activities_activities_activities_activities_df.head()

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español


#### 2.3.1.3 Take unique, non null and non zero latitudes and longitudes, to convert to address.

In [127]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [128]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [129]:
import aiohttp
import asyncio

async def get_address(lat: float, lon: float, api_key: str) -> str:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()
            
            if data.get("status") == "OK":
                address = data["results"][0]["formatted_address"]
                return address
            else:
                return "Address not found"

# Example usage
async def get_adresses_async(lat_lon_df):
    api_key = GOOGLE_API_KEY
    lat_lon_tasks = [get_address(lat, lon, api_key) for lat, lon in lat_lon_df.itertuples(index=False, name=None)]
    addresses = await asyncio.gather(*lat_lon_tasks)
    return addresses


In [130]:
not_nan = activities_df[["latitude", "longitude"]].notna().all(axis=1)
unique_combinations = activities_df.loc[not_nan,["latitude","longitude"]].drop_duplicates()

In [131]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [132]:
unique_combinations["lat_lon"] = list(zip(unique_combinations.latitude, unique_combinations.longitude))
activities_df["lat_lon"] = list(zip(activities_df.latitude, activities_df.longitude))

In [133]:
unique_combinations["address"] = await get_adresses_async(unique_combinations[["latitude", "longitude"]])

In [134]:
lat_lon_to_address_ampping = dict(zip(unique_combinations["lat_lon"], unique_combinations["address"]))

In [135]:
activities_df["address"] = activities_df["lat_lon"].map(lat_lon_to_address_ampping)
activities_df.drop(columns="lat_lon",inplace=True)

In [136]:
activities_df.head(5)

,query_date,activity_date_range_start,activity_date_range_end,activity_name,description,url,image,available_days,available_times,duration,latitude,longitude,price,currency,category,spanish,address
0,2024-11-02 14:13:58.613494,2024-11-27,2024-12-03,Entrada a La Pedrera,Adentraos en el mundo arquitectónico de Gaudí ...,www.civitatis.com/es/barcelona/entrada-la-pedr...,www.civitatis.com/f/espana/barcelona/entrada-l...,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",1h 30m,41.395187,2.161758,4.00,EUR,Entradas,Español,"Pg. de Gràcia, 92, L'Eixample, 08008 Barcelona..."
1,2024-11-02 14:13:58.618562,2024-11-27,2024-12-03,Tour en tuk tuk por Barcelona,¿Os gustaría visitar las maravillas de Gaudí d...,www.civitatis.com/es/barcelona/tour-tuk-tuk-ba...,www.civitatis.com/f/espana/barcelona/tour-tuk-...,"[27, 03, 30, 28, 02, 29, 01]","[[10:00, 11:00, 12:15, 13:15, 16:00, 16:30, 17...",1 -4h,41.3944849,2.1816996,17.80,EUR,Visitas guiadas y free tours,Español,"Estació del Nord, L'Eixample, 08013 Barcelona,..."
2,2024-11-02 14:13:58.622561,2024-11-27,2024-12-03,Paseo en catamarán al atardecer con música en ...,Contempla el skyline de Barcelona mientras dis...,www.civitatis.com/es/barcelona/paseo-catamaran...,www.civitatis.com/f/espana/barcelona/paseo-cat...,"[28, 27, 03, 01, 29, 30, 02]","[[16:30], [16:30], [16:30], [16:30], [16:30], ...",1h 30m,41.37495867288118,2.17849589524371,7.65,EUR,Paseos en barco,Español,"Moll de les Drassanes, 3P, Ciutat Vella, 08039..."
3,2024-11-02 14:13:58.625561,2024-11-27,2024-12-03,Teleférico de Montjuïc,Con esta entrada al Teleférico de Montjuïc pod...,www.civitatis.com/es/barcelona/billete-telefer...,www.civitatis.com/f/espana/barcelona/billete-t...,"[27, 01, 30, 29, 28, 03, 02]","[[10:00], [10:00], [10:00], [10:00], [10:00], ...",None,41.368762,2.163434,3.20,EUR,Entradas,None,"Avinguda Miramar, 30, Sants-Montjuïc, 08038 Ba..."
4,2024-11-02 14:13:58.630562,2024-11-27,2024-12-03,Free tour por el Parque Güell,En este free tour por el Parque Güell conocere...,www.civitatis.com/es/barcelona/free-tour-parqu...,www.civitatis.com/f/espana/barcelona/free-tour...,"[02, 29, 28, 03, 27]","[[11:30], [11:30], [11:30], [9:30, 11:30], [11...",1h 30m,41.41508351,2.154768947,2.00,EUR,Visitas guiadas y free tours,Español,"Ctra. del Carmel, 23, Horta-Guinardó, 08024 Ba..."


#### 2.3.1.4 Format the available dates and times

Available dates and times for each activity will have to go to another table to ensure database normalisation. To do that, it will be necessary to explode the df twice. 

Then, to complete the date of availability, some rules for transformation have to be established:

In [239]:
availability_activities = activities_df[["available_days","available_times","activity_name","activity_date_range_start","activity_date_range_end"]]
availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)

C:\Users\Miguel\AppData\Local\Temp\ipykernel_12864\3428466598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  availability_activities["paired"] = availability_activities[["available_days","available_times"]].fillna("-").apply(lambda x: list(zip(x["available_days"], x["available_times"])), axis=1)


In [240]:
availability_activities = availability_activities.explode("paired", ignore_index=True) #.drop(columns=["available_days","available_times"])

In [241]:
availability_activities.head()

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end,paired
0,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(29, [])"
1,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(02, [])"
2,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(01, [])"
3,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(28, [])"
4,"[29, 02, 01, 28, 03, 30, 27]","[[], [], [], [], [], [], []]",Entrada a La Pedrera,2024-11-27,2024-12-03,"(03, [])"


Now, there is a table with the 

In [242]:
availability_activities[["available_days", "available_times"]] = pd.DataFrame(availability_activities["paired"].tolist(), index=availability_activities.index)
availability_activities = availability_activities.drop(columns="paired")
availability_activities

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end
0,29,[],Entrada a La Pedrera,2024-11-27,2024-12-03
1,02,[],Entrada a La Pedrera,2024-11-27,2024-12-03
2,01,[],Entrada a La Pedrera,2024-11-27,2024-12-03
3,28,[],Entrada a La Pedrera,2024-11-27,2024-12-03
4,03,[],Entrada a La Pedrera,2024-11-27,2024-12-03
...,...,...,...,...,...
1272,-,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09
1273,-,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09
1274,-,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09
1275,-,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09


In [ ]:
availability_activities = availability_activities.explode("available_times")

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end
0,29,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
1,02,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
2,01,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
3,28,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
4,03,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
...,...,...,...,...,...
1272,-,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09
1273,-,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09
1274,-,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09
1275,-,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09


In [246]:
availability_activities["available_days"] = availability_activities["available_days"].replace("-",np.nan)
availability_activities

,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end
0,29,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
1,02,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
2,01,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
3,28,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
4,03,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03
...,...,...,...,...,...
1272,NaN,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09
1273,NaN,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09
1274,NaN,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09
1275,NaN,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09


In [ ]:
def calculate_adjusted_date(row):

    row["activity_date_range_start"] = pd.to_datetime(row["activity_date_range_start"])
    row["activity_date_range_end"] = pd.to_datetime(row["activity_date_range_end"])


    if pd.isna(row["available_days"]):
        return pd.NaT
    day = int(row["available_days"])  # Get the day from available_days
    # Check if the day is greater than the day of activity_date_range_end
    if day > row["activity_date_range_end"].day:
        # Use activity_date_range_end's month and year with the day from available_days
        return row["activity_date_range_start"].replace(day=day)
    else:
        # Use activity_date_range_start's month and year with the day from available_days
        return row["activity_date_range_end"].replace(day=day)

# Apply the custom function to each row
availability_activities["available_date"] = availability_activities.apply(calculate_adjusted_date, axis=1)

availability_activities.drop(columns="available_days",inplace=True)


,available_days,available_times,activity_name,activity_date_range_start,activity_date_range_end,adjusted_date
0,29,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03,2024-11-29
1,02,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03,2024-12-02
2,01,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03,2024-12-01
3,28,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03,2024-11-28
4,03,NaN,Entrada a La Pedrera,2024-11-27,2024-12-03,2024-12-03
...,...,...,...,...,...,...
1272,NaN,-,Tour de los misterios y leyendas de Barcelona,2024-12-03,2024-12-09,NaT
1273,NaN,-,Entrada al Spotify Camp Nou,2024-12-03,2024-12-09,NaT
1274,NaN,-,Free tour de los misterios y leyendas del Barr...,2024-12-03,2024-12-09,NaT
1275,NaN,-,"Excursión a Gerona, Figueras y Museo Dalí",2024-12-03,2024-12-09,NaT


In [234]:
availability_activities["adjusted_date"] = np.where(
    availability_activities["available_days"].isna(), 
    pd.NaT,  # If available_days is NaN, keep adjusted_date as NaN
    np.where(
        availability_activities["available_days"] > availability_activities["activity_date_range_end"].dt.day,
        # If available_days is greater, use activity_date_range_end's month
        availability_activities["activity_date_range_start"].apply(lambda x: x.replace(day=availability_activities["available_days"])),
        # Otherwise, use activity_date_range_start's month
        availability_activities["activity_date_range_end"].apply(lambda x: x.replace(day=availability_activities["available_days"]))
    )
)
availability_activities

ValueError: value must be an integer, received <class 'pandas.core.series.Series'> for day

In [ ]:

# Drop the helper column if not needed
activities_df = activities_df.drop(columns="last_day_of_month_start")

print(activities_df[["available_days", "activity_date_range_start", "activity_date_range_end", "adjusted_date"]])


In [194]:
pd.to_datetime(availability_activities["activity_date_range_end"]).dt.day

0       3
1       3
2       3
3       3
4       3
       ..
1272    9
1273    9
1274    9
1275    9
1276    9
Name: activity_date_range_end, Length: 1277, dtype: int32

In [200]:
availability_activities["available_days"] = availability_activities["available_days"].replace("-",np.nan)

In [201]:
np.where(pd.to_datetime(availability_activities["activity_date_range_end"]).dt.day < availability_activities["available_days"].astype(float),)

(array([  0,   3,   5,   6,   7,   9,  10,  12,  14,  15,  18,  19,  21,
         23,  24,  25,  29,  30,  32,  34,  37,  38,  39,  40,  41,  42,
         45,  46,  47,  49,  50,  52,  53,  55,  56,  58,  59,  61,  62,
         65,  66,  70,  71,  72,  73,  74,  76,  80,  82,  84,  85,  86,
         87,  90,  91,  93,  96,  97,  99, 101, 102, 104, 107, 108, 110,
        111, 114, 115, 117, 118, 120, 121, 123, 125, 127, 128, 130, 133,
        135, 136, 137, 138, 140, 141, 144, 145, 146, 149, 151, 152, 153,
        155, 159, 160, 161, 163, 164, 165, 166, 168, 169, 171, 172, 174,
        176, 179, 182, 183, 185, 188, 189, 190, 191, 192, 193, 196, 199,
        201, 202, 204, 207, 208, 209, 212, 213, 215, 218, 219, 220, 223,
        224, 225, 226, 229, 230, 232, 233, 236, 237, 240, 242, 245, 246,
        247, 249, 250, 251, 253, 254, 257, 259, 260, 262, 266, 267, 268,
        269, 271, 272, 275, 277, 278, 279, 280, 282, 284, 285, 288, 290,
        291, 292, 295, 296, 297, 300, 302, 303, 304

In [ ]:
availability_activities["available_date"] = 